# 나만의 자비스! 음성 비서 만들기
OpenAI의 ChatGPT의 **Function Calling** 기능을 활용하면, 아이언맨의 자비스와 같은 인공지능 비서를 만들 수 있습니다.  

![](assets/stt_chatgpt_tts.png)

한국어 대화도 가능합니다. 실시간 대화 기능을 구현하기 위해서 다음의 기술을 활용할 수 있습니다.  

1. 사용자의 음성은 Microsoft Azure가 제공하는 STT(Speech to Text) 서비스를 이용하여 텍스트로 변환합니다.
2. 텍스트는 ChatGPT가 제공하는 Function Calling 기능을 활용하여 이미 작성되어진 API 또는 코드를 선택적으로 수행합니다.
3. 수행 결과는 다시 Microsoft Azure가 제공하는 TTS(Text to Speech)를 이용하여 대답할 수 있습니다.

> **Notes**
>- 이 코드는 Python 3.10.11 커널에서 테스트 하였습니다.
>- 아래 코드는 PoC 용도입니다. 완벽한 코드가 아니므로 참고용으로 활용할 수 있습니다.
>- 해당 STT 및 TTS 기능을 활용하기 위해서는 마이크와 스피커와 같은 Hardware를 필요로 합니다. 개발환경이 컨테이너 기반일 경우, 정상 수행되지 않을 수 있습니다.

In [1]:
# 데스크톱(로컬)에서 실행하는 경우 입니다. 아래와 같이 라이브러리 재설치가 필요할 수 있습니다.
# 먼저 파이썬 런타임을 설치해야 합니다. https://www.python.org/downloads/
# !pip install azure-cognitiveservices-speech
# !pip install azure-identity
# !pip install -r ../requirements.txt

실행에 필요한 환경 변수를 가져옵니다. 아래에 필요한 정보를 미리 .env 파일에 저장합니다.

1. Azure OpenAI API 정보
2. Azure Speech API 정보
3. Kakao REST API 정보 (https://developers.kakao.com/console/app)
4. openweathermap API 정보 (https://openweathermap.org/current)

In [2]:
import azure.cognitiveservices.speech as speechsdk
import os
import openai
import json
import requests
import pytz
from urllib import parse
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2023-06-01-preview"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model=os.getenv('DEPLOYMENT_NAME')

# Azure OpenAI resource 정보를 설정합니다. .env 파일에 설정된 정보를 사용합니다.
openai.api_type     = os.getenv("OPENAI_API_TYPE")
openai.api_key      = os.getenv("OPENAI_API_KEY")
openai.api_base     = os.getenv("OPENAI_API_BASE")
openai.api_version  = os.getenv("OPENAI_API_VERSION")       # API 버전은 "2023-07-01-preview" 부터 사용 가능합니다.
deployment_id       = os.getenv("DEPLOYMENT_NAME_16K")      # Azure OpenAI resource의 deployment id를 입력합니다.
KAKAO_API_KEY       = os.getenv("KAKAO_REST_API_KEY")       # KAKAO REST API 키입니다.
speech_key          = os.getenv("AZURE_SPEECH_KEY")         # Azure Speech Service의 Speech Key입니다.
speech_region       = os.getenv("AZURE_SPEECH_REGION")      # Azure Speech Service의 서비스 지역입니다.
speech_language     = os.getenv("AZURE_SPEECH_LANGUAGE")    # Azure Speech Service의 서비스 언어입니다.
WEATHER_API_KEY     = os.getenv("WEATHER_API_KEY")          # 날씨 정보를 가져오기 위한 openweathermap API 키입니다.

### Azure Speech To Text (STT) 엔진으로 사용자 명령을 텍스트로 수집

In [3]:
# Azure Cognitive Speech to Text 함수
def stt():
    # Creates a recognizer with the given settings
    # Azure STT & TTS API key
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region, speech_recognition_language='ko-KR')
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

    print("말씀 하세요~")

    # Starts speech recognition, and returns after a single utterance is recognized. The end of a
    # single utterance is determined by listening for silence at the end or until a maximum of 15
    # seconds of audio is processed.
    result = speech_recognizer.recognize_once()

    # Checks result.
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("음성인식결과: {}".format(result.text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("일치하는 음성이 없습니다.: {}".format(result.no_match_details))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("음성 인식이 취소되었습니다.: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(
                cancellation_details.err+or_details))
    return result

### Azure Text To Speech (TTS) 엔진으로 텍스트를 읽음

In [4]:
# Azure Cognitive Text to Speech 함수
def tts(input):
    # Set the voice name, refer to https://learn.microsoft.com/ko-kr/azure/ai-services/speech-service/language-support?tabs=tts for full list.
    # speech_config.speech_synthesis_voice_name = "ko-KR-InJoonNeural"
    # Creates a synthesizer with the given settings
    # Azure STT & TTS API key
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_synthesis_voice_name = "ko-KR-SeoHyeonNeural"
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

    # Synthesizes the received text to speech.
    result = speech_synthesizer.speak_text_async(input).get()

    # Checks result.
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized to speaker for text [{}]".format(input))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Speech synthesis canceled: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(
                    cancellation_details.error_details))
        print("Did you update the subscription info?")

### 지명 기반의 위경도 좌표 위치 찾기 (Kakao API 활용)

In [5]:
# Kakao REST API를 활용하는 함수 (주소를 위경도 좌표로 변환 포함)
headers = {
    "Authorization": "KakaoAK " + KAKAO_API_KEY,
    "Content-Type": "application/json",
}    

# Kakao 키워드 기반 위경도 좌표 찾기
def get_location_xy(keyword="한국마이크로소프트"):
    params = {
        "query": keyword
    }
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?" + parse.urlencode(params)
    response = requests.get(url, headers=headers)
    return (response.json()["documents"][0])

# Convert from seconds to hours, minutes and seconds
def convert_second(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    
    return "%d시간 %d분 %d초" % (hour, minutes, seconds)

# Convert from meter to kilometer
def convert_meter(meter):
    return str(round(meter / 1000, 2))


### Function 1. Kakao Mobility 길찾기 API

In [6]:
# Kakao 길찾기 API
def get_directions(origin, destination, waypoints="", priority="RECOMMEND", car_fuel="GASOLINE", car_hipass="true", alternatives="false", road_details="false"):
    # 키워드 기반 위경도 좌표 정보 수집
    xy_info = get_location_xy(origin)
    origin_xy_info = xy_info["x"] + "," + xy_info["y"] + ",name=" + xy_info["place_name"]
    xy_info = get_location_xy(destination)
    destin_xy_info = xy_info["x"] + "," + xy_info["y"] + ",name=" + xy_info["place_name"]
    
    params = {
        "origin": origin_xy_info,
        "destination": destin_xy_info,
        "waypoints": waypoints,
        "priority": priority,
        "car_fuel": car_fuel,
        "car_hipass": car_hipass,
        "alternatives": alternatives,
        "road_details": road_details,
    }
    url = "https://apis-navi.kakaomobility.com/v1/directions?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["routes"][0]["summary"]
    return_data = {
        "origin_name": response_summary["origin"]["name"],
        "destination_name": response_summary["destination"]["name"],
        "taxi_fare": response_summary["fare"]["taxi"],
        "toll_fare": response_summary["fare"]["toll"],
        "distance": convert_meter(response_summary["distance"]) + "km",
        "duration": convert_second(response_summary["duration"]),
    }
    
    return json.dumps(return_data)

### Function 2. Kakao Mobility 미래 운행 시간 길찾기 API

In [7]:
# Kakao 미래 운행 시간 기준 길찾기 API
def get_future_directions(origin, destination, departure_time, waypoints="", priority="RECOMMEND", car_fuel="GASOLINE", car_hipass="true", alternatives="false", road_details="false"):
    # 키워드 기반 위경도 좌표 정보 수집
    xy_info = get_location_xy(origin)
    origin_xy_info = xy_info["x"] + "," + xy_info["y"] + ",name=" + xy_info["place_name"]
    xy_info = get_location_xy(destination)
    destin_xy_info = xy_info["x"] + "," + xy_info["y"] + ",name=" + xy_info["place_name"]
    
    # 시간 포맷팅을 API에 맞게 수정 요청합니다. (이미 Function Calling 함수내 파라미터 값에 정의하였기에 실행할 필요가 없음)
    # print("원래 시간: ")
    # print(departure_time)
    # time_format = openai.ChatCompletion.create(
    #     deployment_id=deployment_id,
    #     messages=[
    #         {"role": "system", "content": "You are an agent that converts a date or time value to a format of the form: %Y%m%d%H%M"},
    #         {"role": "user", "content": "2023-11-26T15:30:00"},
    #         {"role": "assistant", "content": "202311261530"},
    #         {"role": "user", "content": departure_time}
    #     ]
    # )
    # print("변경 포맷: ")
    # print(time_format["choices"][0]["message"]["content"])
    
    params = {
        "origin": origin_xy_info,
        "destination": destin_xy_info,
        "waypoints": waypoints,
        "priority": priority,
        "car_fuel": car_fuel,
        "car_hipass": car_hipass,
        "alternatives": alternatives,
        "road_details": road_details,
        "departure_time": departure_time,
    }
    url = "https://apis-navi.kakaomobility.com/v1/future/directions?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["routes"][0]["summary"]
    return_data = {
        "origin_name": response_summary["origin"]["name"],
        "destination_name": response_summary["destination"]["name"],
        "taxi_fare": response_summary["fare"]["taxi"],
        "toll_fare": response_summary["fare"]["toll"],
        "distance": convert_meter(response_summary["distance"]) + "km",
        "duration": convert_second(response_summary["duration"]),
    }
    
    return json.dumps(return_data)


### Function 3. 실시간 지역 시간 수집

In [8]:
def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%Y%m%d%H%M")

        return current_time
    except:
        return "죄송합니다. 해당 지역의 TimeZone을 찾을 수 없습니다."

### Function 4. 실시간 지역 날씨 수집

In [9]:
# 특정 지역의 날씨를 가져오는 함수
def get_current_weather(location="서울시청"):
    xy_info = get_location_xy(location)
    params = {
        "lat": xy_info["y"],
        "lon": xy_info["x"],
        "units": "metric",
        "lang":  "en",
        "appid": WEATHER_API_KEY
    }
    url = "https://api.openweathermap.org/data/2.5/weather?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)

    return_data = {
        "Weather_main": response.json()["weather"][0]["main"],
        "Weather_description": response.json()["weather"][0]["description"],
        "Temperature_Celsius": response.json()["main"]["temp"],
        "Humidity": response.json()["main"]["humidity"],
        "Cloudiness": response.json()["clouds"]["all"]
    }

    return json.dumps(return_data)

#### OpenWeatherMap 날씨 API 동작 체크

In [10]:
# 날씨 정보를 제대로 수집하는지를 확인합니다.
response = get_current_weather("서울")
print(response)

{"Weather_main": "Clear", "Weather_description": "clear sky", "Temperature_Celsius": 15.83, "Humidity": 94, "Cloudiness": 0}


### OpenAI Function Calling에서 활용하려는 함수들을 정의

In [11]:
functions = [
        {
            "name": "get_directions",
            "description": "API to search routes based on origin and destination information",
            # "description": "출발지와 도착지 정보를 기반으로 경로 검색하는 API",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},                    
                },
                "required": ["origin", "destination"],
            },
        },
        {
            "name": "get_future_directions",
            "description": "API to search routes based on origin and destination information based on future departure_time",
            # "description": "출발지와 도착지 정보를 미래 시간 기반으로 경로 검색하는 API",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},
                    "departure_time": {
                        "type": "string",
                        "description": "The time format of the given time must be converted to %Y%m%d%H%M format. If there is no year information, 2023 is used as the default. ",
                    },
                },
                "required": ["origin", "destination", "departure_time"],
            },
        },
        {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            # "description": "이 지역의 현재 시간을 알려줘.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like Asia/Seoul, America/New_York, Asia/Bangkok, Europe/London"
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "get_current_weather",
            "description": "Get the current weather information in a given location",
            # "description": "이 지역의 현재 날씨를 알려줘.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name. City names should be in a format like 서울, 부산, 속초, 대구"
                    }
                },
                "required": ["location"],
            },
        },
        
    ]

available_functions = {
            "get_directions": get_directions,
            "get_future_directions": get_future_directions,
            "get_current_time": get_current_time,
            "get_current_weather": get_current_weather,
        } 

#### 함수에 제공되는 매개변수가 맞는지 검수하는 함수

In [12]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

#### OpenAI GPT 모델을 이용하여 사용자 의도에 맞는 함수를 자동 선택하고 실행하는 함수

In [13]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT
    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # print("Recommended Function call:")
        # print(response_message.get("function_call"))
        # print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        # print("Output of function call:")
        # print(function_response)
        # print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # function_name 값에 따른 분기 처리
        if function_name == "get_directions" or function_name == "get_future_directions":
            messages.append(
                {"role": "system", "content": "You are a bot that guides you through car routes. When the user provides the origin and destination name, you provides summary route guidance information.",}
            )
        elif function_name == "get_current_weather":
            messages.append(
                {"role": "system", "content": "You are an agent that tells the user about the weather. You describe based on the given data and do not judge and create other sentences."},
            )
        elif function_name == "get_current_time":
            messages.append(
                {"role": "system", "content": "You are a bot that tells the world time. You describe based on the given data and do not judge and create other sentences."},
            )
        else :
            messages.append(
                {"role": "system", "content": "You are an AI assistant that helps people find information. The answer must be judged and answered based on factual data. Please use simple expressions as much as possible."},
            )
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        # print("Messages in second request:")
        # for message in messages:
        # print(messages)
        # print(json.dumps(messages, ensure_ascii=False, indent=4))

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id,
            temperature=0
        )  # get a new response from GPT where it can see the function response
        
        # print("Second Call: ")
        # print(second_response)
        # print()

        return second_response

#### GPT 모델에 자연어 기반 질의하는 함수
해당 실습에서는 Function Calling에 해당되지 않는 요청하면 종료합니다.

In [14]:
def gpt(input):
    messages = [
        {"role": "user", "content": input}
    ]
    assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
    # assistant_response 값이 비어있을 경우
    if not assistant_response:
        return "제가 답변 드릴 수 있는 질문이 아닙니다. 다시 질문해주세요."
    else:
        content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
        content = content.replace("\\n", "\n").replace("\\\"", "\"")
        return content


GPT에 질의하여 각 기능이 정상적으로 동작하는지 살펴 봅니다.

In [15]:
future_time = ""
# future_time = "2023년 12월 21일 18시에"
origin_name = "광화문역"
destin_name = "강남역"
query = f"{future_time} {origin_name}에서 {destin_name}까지 얼마나 걸려?"
query = "서울 날씨 어때?"
# query = "사랑해"

print(gpt(query))

"서울의 현재 날씨는 맑은 하늘이며, 기온은 섭씨 15.83도입니다. 습도는 94%이고 구름이 거의 없습니다."


In [17]:
# if __name__ == "__main__":
tts("날씨, 길찾기, 시간을 물어보세요.")

while True:
    result_stt = stt().text
    print(result_stt)
    if(result_stt == ""):
        # 음성 인식 실패
        print("음성 인식 실패")
        tts("음성인식에 실패했습니다. 다시 말씀해 주세요.")
    elif(result_stt == "나가기." or result_stt == "종료."):
        print("대화 종료")
        break
    else:
        # 음성인식 성공
        result_gpt = gpt(result_stt)
        tts(result_gpt)

Speech synthesized to speaker for text [날씨, 길찾기, 시간을 물어보세요.]
말씀 하세요~
음성인식결과: 강남역에서 광화문역까지 얼마나 걸려?
강남역에서 광화문역까지 얼마나 걸려?
Speech synthesized to speaker for text ["강남역에서 광화문역까지는 약 11.5km 거리이며, 차로 약 23분 1초 소요됩니다. 택시 요금은 약 18,000원이며, 톨비는 없습니다."]
말씀 하세요~
음성인식결과: 부산 날씨 알려줘.
부산 날씨 알려줘.
Speech synthesized to speaker for text ["부산의 현재 날씨는 흐린 하늘과 구름이 많이 끼어있습니다. 기온은 약 19.98도이며 습도는 88%입니다. 구름이 100%로 많이 끼어있습니다."]
말씀 하세요~
음성인식결과: 서울 지금 몇 시야?
서울 지금 몇 시야?
Speech synthesized to speaker for text ["현재 서울의 시간은 2023년 9월 22일 오전 5시 55분입니다."]
말씀 하세요~
음성인식결과: 나가기.
나가기.
대화 종료


이제부터는 **여러분의 API** 를 하나씩 추가해 보세요.
수고하셨습니다.